In [ ]:
# Tests for methods that merge horizons
import sys
import warnings
import math
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb import Field, Horizon, plot_image

In [ ]:
""" You can manage notebook execution kwargs which relates to cube and horizon for the test:

CUBE_PATH : str or None
    Path to an existed seismic cube.
HORIZON_PATH : str or None
    Path to an existed seismic horizon.

Visualizations in saved execution notebooks are controlled with:

SCALE : int
    Figures scale.
SHOW_FIGURES : bool
    Whether to show additional figures.
    Showing some figures can be useful for finding out the reason for the failure of tests.

Text outputs controlled with:

VERBOSE : bool
    Whether to print information about successful tests during the execution of the cycles.
"""
# Tests parameters
CUBE_PATH = './horizon_test_files/tmp/test_cube.sgy'
HORIZON_PATH = './horizon_test_files/tmp/test_horizon'

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

# Output parameters
VERBOSE = True

In [ ]:
def test_horizons_equality(horizon_1, horizon_2, test_name):
    " Check two horizons equality."
    assert np.array_equal(horizon_1.full_matrix, horizon_2.full_matrix), f"{test_name} test failed"
    assert np.array_equal(horizon_1.points, horizon_2.points), f"{test_name} test failed"

In [ ]:
%%time
field = Field(CUBE_PATH)

horizon = Horizon(HORIZON_PATH, field=field)
field.load_labels({'horizons': horizon})

horizon.show(show=SHOW_FIGURES)

In [ ]:
message = ""

# Merge two horizons

In [ ]:
configs = [
#   (first horizon border, second horizon border, verify_merge code)
    (len(horizon.points)//2, len(horizon.points)//2-len(horizon.points)//10, 4),        # overlap
    (len(horizon.points)//10, len(horizon.points)//10, 3),                              # close without overlap
    (len(horizon.points)//10, len(horizon.points)//2, 1),                               # spatially_far
    (len(horizon.points)//10, len(horizon.points)//10-len(horizon.points)//20, 0)       # height-wise far
]

In [ ]:
%%time
# Horizon.*_merge methods

for (border_1, border_2, code) in configs:
    # Make new horizons
    horizon_1 = Horizon(horizon.points[:border_1, :],
                        field=field, name="horizon_1")

    horizon_2 = Horizon(horizon.points[border_2:, :],
                        field=field, name="horizon_2")

    if code == 0: # make horizons height-wise far
        horizon_2.points[:, 2] += 50

    # Check merge code
    error_message = f"merge two horizons with code {code} test failed"
    assert horizon_1.verify_merge(horizon_2, adjacency=1) == code, error_message

    # Merge and check horizons if they are mergeable
    if code == 4:
        merged_horizon = Horizon.overlap_merge(horizon_1, horizon_2)

        assert horizon.equal(merged_horizon), "merge overlapped horizons failed"

    elif code == 3:
        merged_horizon = horizon_1.adjacent_merge(horizon_2)

        assert horizon.equal(merged_horizon), "merge close without overlap horizons failed"

    if VERBOSE:
        current_message = f"The two horizons merge test with merge code {code} was successfully passed.\n"
        print(current_message)
        message += current_message

In [ ]:
%%time
# average_horizons method

for (border_1, border_2, code) in configs:
    # Make new horizons
    horizon_1 = Horizon(horizon.points[:border_1, :],
                        field=field, name="horizon_1")

    horizon_2 = Horizon(horizon.points[border_2:, :],
                        field=field, name="horizon_2")

    if code == 0: # make horizons height-wise far
        horizon_2.points[:, 2] += 50

    # Get averaged horizons points
    averaged_horizon_info = Horizon.average_horizons([horizon_1, horizon_2])

    averaged_horizon_points = Horizon.matrix_to_points(
        averaged_horizon_info[1]["matrix"]
    ).astype(int)

    # Calculate averaged horizons points with horizons points
    points_1 = pd.DataFrame(horizon_1.points, columns=["i","x","h"]).set_index(["i", "x"])
    points_2 = pd.DataFrame(horizon_2.points, columns=["i","x","h"]).set_index(["i", "x"])

    # Overlapping part
    intersection_idxs = points_1.index.intersection(points_2.index)

    intersection_average = (points_1.loc[intersection_idxs] + points_2.loc[intersection_idxs]) // 2
    intersection_average = intersection_average.astype(int)

    # Not-overlapping part
    non_intersected_points_1 = points_1.drop(intersection_idxs)
    non_intersected_points_2 = points_2.drop(intersection_idxs)

    # Get averaged points
    averaged_points = pd.concat(
        [non_intersected_points_1, non_intersected_points_2, intersection_average]
    )

    averaged_points.sort_index(inplace=True)
    averaged_points = averaged_points.reset_index().to_numpy()

    assert np.array_equal(
        averaged_horizon_points, averaged_points
    ), f"average_horizons for two horizons with code {code} test failed"

    if VERBOSE:
        current_message = f"The two horizons average test with merge code {code} was successfully passed.\n"
        print(current_message)
        message += current_message

# Merge multiple horizons

We split a horizon in multiple parts, merge them into one horizon and compare it with original one.

In [ ]:
horizon.filter()

zero_traces = np.tril(horizon.full_matrix).astype(bool)
horizon.filter(zero_traces)

horizon.filter_disconnected_regions()

horizon.show(show=SHOW_FIGURES, scale=SCALE)

In [ ]:
def split_and_merge_horizon_test(horizon, crop_shape, overlap, adjacency):
    """ Split a horizon in crops with overlap and merge them.

    Merged horizon compared with the original.

    Parameters
    ----------
    horizon : instance of the `Horizon` class
        Seismic horizon.
    crop_shape : sequence of two integers
        Crop shape (ilines, xlines).
    overlap : sequence of two integers
        Crops overlap by (ilines, xlines).
    adjacency : int
        Margin to consider horizons to be close (spatially).
    """
    current_message = ""
    borders = []

    # Get splitting borders
    for axis in range(2):
        crop_num_ = math.ceil(horizon.full_matrix.shape[axis]/(crop_shape[axis] - overlap[axis]))
        axis_borders = [(i*(crop_shape[axis] - overlap[axis]),
                        (i*(crop_shape[axis] - overlap[axis]) + crop_shape[axis]))
                        for i in range(crop_num_)]

        borders.append(axis_borders)

    horizons = []

    # Split a horizon
    for i_line_border_num in range(0, len(borders[0])):
        for x_line_border_num in range(0, len(borders[1])):
            horizon_i_borders = borders[0][i_line_border_num]
            horizon_x_borders = borders[1][x_line_border_num]

            cutted_horizon_matrix = horizon.full_matrix[horizon_i_borders[0]:horizon_i_borders[1],
                                                        horizon_x_borders[0]:horizon_x_borders[1]]

            if not np.all(cutted_horizon_matrix == horizon.FILL_VALUE):
                horizon_ = Horizon(cutted_horizon_matrix, force_format="matrix",
                                   i_min=horizon_i_borders[0], x_min=horizon_x_borders[0],
                                   field=field,
                                   name=f'A_horizon_{i_line_border_num*x_line_border_num}')

                horizons.append(horizon_)

    merged_ = np.zeros_like(horizons[0].full_matrix)

    for horizon_ in horizons:
        merged_ += (horizon_.full_matrix > 0).astype(np.int32)

    # Horizon merge: inplace
    horizons, _ = Horizon.merge_list(horizons, adjacency=adjacency, mean_threshold=5.)
    merged_horizon = horizons[-1]

    params = f"crop_shape={crop_shape}; overlap={overlap}; adjacency={adjacency}"


    # Take a look at horizon parts summation
    if SHOW_FIGURES:
        plot_image([merged_, merged_horizon.full_matrix], separate=True,
                   scale=SCALE, suptitle_label=params, suptitle_y=0.9,
                   colorbar=True, fontsize=14)

    # Asserts
    if (overlap[0] >= 0) and (overlap[1] >= 0):
        n_missing = 0
        assert horizon.equal(merged_horizon), f"""Merge list with ({params}) failed!"""

    else:
        n_missing = len(horizon) - (merged_ > 0).sum()
        error_message = f"Merge list with ({params}) failed!"
        assert horizon.equal(merged_horizon, threshold_missing=n_missing + 1), error_message

    if VERBOSE:
        current_message = f"The multiple horizons merge test with params {params} was successfully passed.\n"
        print(current_message)

    return current_message

In [ ]:
overlaps = [(i, j) for i in range(-3, 4) for j in range(-3, 4)]

SHOW_FIGURES = False

crop_shapes = [
    (horizon.full_matrix.shape[0]//20+1, horizon.full_matrix.shape[1]//20+1), # Split horizon into small square crops
    (horizon.full_matrix.shape[0]//10+1, horizon.full_matrix.shape[1]//10+1), # Split horizon into medium square crops
    (horizon.full_matrix.shape[0]//5+1, horizon.full_matrix.shape[1]//5+1),   # Split horizon into big square crops
    (horizon.full_matrix.shape[0], horizon.full_matrix.shape[1]//20+1),       # Split horizon by i_lines
    (horizon.full_matrix.shape[0]//20+1, horizon.full_matrix.shape[1])        # Split horizon by x_lines
]

for crop_shape in crop_shapes:
    for overlap in overlaps:
        min_ = min(overlap)
        adjacency = 0 if min_>0 else -min_ + 1
        message += split_and_merge_horizon_test(horizon=horizon, crop_shape=crop_shape,
                                                overlap=overlap, adjacency=adjacency)

# Extract

In [ ]:
%%time
valid_traces = 1 - field.zero_traces
mask = np.zeros(field.shape, dtype=np.int32)
for horizon in field.labels:
    mask[horizon.points[:, 0], horizon.points[:, 1], horizon.points[:, 2]] = 1


origin = np.array([0, 0, 0])

# No chunks

In [ ]:
horizons = Horizon.from_mask(mask.copy(), field=field, origin=origin)
extracted_horizon = horizons[-1]
horizon.equal(extracted_horizon)

# With chunks

In [ ]:
def chunked_extraction_test(mask, true_horizon, valid_traces, origin, step, overlap):
    current_message = ""
    params = f'step={step}; overlap={overlap}'
    i_step = x_step = step
    i_overlap = x_overlap = overlap

    # Make an iterator of subvolumes
    iterator = []
    for i_start in range(0, mask.shape[0], i_step - i_overlap):
        for x_start in range(0, mask.shape[1], x_step - x_overlap):
            i_end = min(i_start + i_step, mask.shape[0])
            x_end = min(x_start + x_step, mask.shape[1])

            slices = (slice(i_start, i_end), slice(x_start, x_end), slice(None))

            if valid_traces[slices[:2]].sum() > 0:
                iterator.append(slices)


    # Extract surfaces from each subvolume
    horizons = []
    for subvolume_slices in iterator:
        subvolume = mask[subvolume_slices].copy()
        subvolume_origin = origin + [slc.start or 0 for slc in subvolume_slices]

        horizons_, _ = Horizon.extract_from_mask(subvolume, field=field, origin=subvolume_origin,
                                                 minsize=0, verbose=False)
        horizons.extend(horizons_)


    # Merge horizons from chunks
    horizons, _ = Horizon.merge_list(horizons, mean_threshold=0.5,
                                               max_threshold=0.2, adjacency=0)
    extracted_horizon = horizons[-1]
    assert true_horizon.equal(extracted_horizon), f"Failed extraction with chunks: {params}"

    if VERBOSE:
        current_message = f"The chunks test with params {params} was successfully passed.\n"
        print(current_message)
    return current_message

In [ ]:
for step in [30, 50, 100]:
    for overlap in [5, 10, 15]:
        message += chunked_extraction_test(mask.copy(), true_horizon=horizon,
                                           valid_traces=valid_traces, origin=origin,
                                           step=step, overlap=overlap)